In [1]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

# 資料作成に使用する最もフィットするモデルはどれかを示すための表・グラフを作るためのプログラム


In [2]:
def return_row_list(input_list):
    sum_of_list = 0
    return_list = []
    for list_child in input_list:
        sum_of_list += len(list_child)
    for list_child in input_list:
        num = int(len(list_child)/sum_of_list*100)
        if(len(list_child) == 0):
            min_data = "NoData"
            max_data = "NoData"
        else:
            min_data = min(list_child)
            max_data = max(list_child)
        string = f"{num}({min_data}-{max_data})"
        return_list.append(string)
    return return_list

In [6]:
csv_directory_path = './tmp_GenerateResources/'

result_of_all = [[], [], []]
list_for_csv = []
for benchmark in benchmarks:
    result_of_benchmark = [[], [], []]
    file_name = benchmark+'.csv'
    file_path = csv_directory_path+file_name
    # ファイルが存在しない場合は処理を飛ばす
    if(os.path.isfile(csv_directory_path+file_name) == False):
        continue
    # 完全に値をとれたもののみで集計するようにしている
    if(benchmark != "cg" and benchmark != "ep" and benchmark != "lu"):
        continue
    
    print(benchmark)
    df = pd.read_csv(file_path, index_col=0)
    columns = df.columns.values
    indices = df.index.values
    for index in indices:
        row = df.loc[index].tolist()
        result_of_benchmark[row.index(min(row))].append(min(row))
        result_of_all[row.index(min(row))].append(min(row))
    list_for_csv.append([benchmark]+return_row_list(result_of_benchmark))
list_for_csv.append(["all"]+return_row_list(result_of_all))

cg
ep
lu


In [7]:
print(list_for_csv)

[['cg', '86(0.0-6.918261603219528)', '13(7.99968321779567-9.589617808669685)', '0(NoData-NoData)'], ['ep', '100(0.0-0.0)', '0(NoData-NoData)', '0(NoData-NoData)'], ['lu', '69(0.0-0.0)', '21(15.23493037122562-17.01274273602921)', '9(0.0-12.247010037505465)'], ['all', '78(0.0-6.918261603219528)', '16(7.99968321779567-17.01274273602921)', '5(0.0-12.247010037505465)']]


In [8]:
csv_head = ["", "線形モデル", "対数モデル", "反比例モデル"]
df_for_csv = pd.DataFrame(list_for_csv, index=csv_head)
df_for_csv

,0,1,2,3
,cg,86(0.0-6.918261603219528),13(7.99968321779567-9.589617808669685),0(NoData-NoData)
線形モデル,ep,100(0.0-0.0),0(NoData-NoData),0(NoData-NoData)
対数モデル,lu,69(0.0-0.0),21(15.23493037122562-17.01274273602921),9(0.0-12.247010037505465)
反比例モデル,all,78(0.0-6.918261603219528),16(7.99968321779567-17.01274273602921),5(0.0-12.247010037505465)
